In [5]:
# install deps
!pip install --quiet alpaca_trade_api pandas numpy matplotlib pyppeteer schedule nest_asyncio

import asyncio, nest_asyncio, time, schedule, os
from pyppeteer import launch
import pandas as pd, requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta, time as dtime
import numpy as np
from alpaca_trade_api.rest import REST, TimeFrame
import datetime as dt

nest_asyncio.apply()

# ─── CONFIG ────────────────────────────────────────────────────────────────
# Alpaca paper credentials
API_KEY    = "PKCD5EQESCEUAEK6P229"                # ← your paper key
API_SECRET = "vMBP4zAug1Qyv8ngPNweTiI3GINrlQS5rtT8LFYC"  # ← your paper secret
BASE_URL   = "https://paper-api.alpaca.markets"     # ← paper endpoint

api = REST(API_KEY, API_SECRET, BASE_URL, api_version="v2")

# Where to save the scraped CSV
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/quant/Everyday_Stocks"
os.makedirs(DRIVE_PATH, exist_ok=True)

# How many to pick & weights
PICK_COUNT = 10
PCT_WEIGHT = 0.10   # 10% pctChange, 90% market cap
MAX_USAGE  = 1    # use up to 70% of buying power each day

# Market hours / schedule
SCRAPE_TIME = "09:55"
BUY_TIME    = "10:00"
SELL_TIME   = "14:00"
WIN_START   = "09:30"
WIN_END     = "16:00"
RATE_SLEEP  = 0  # throttle for Alpaca/minute‐bar pulls

# ─── SCRAPER ────────────────────────────────────────────────────────────────
GAINERS_URL = "https://stockanalysis.com/markets/gainers/"

async def _scrape_top50():
    browser = await launch(headless=True,
                          args=["--no-sandbox","--disable-setuid-sandbox"])
    page = await browser.newPage()
    await page.goto(GAINERS_URL, {"waitUntil":"networkidle2"})
    await page.waitForSelector("table#main-table tbody tr")

    # click 20→50 rows
    span20 = (await page.xpath("//span[normalize-space(text())='20 Rows']"))[0]
    btn20  = await page.evaluateHandle("(el)=>el.closest('button')", span20)
    await btn20.click(); await asyncio.sleep(0.3)
    await page.click("button[title='Show 50 Rows']"); await asyncio.sleep(0.3)

    # wait for exactly 50 rows
    await page.waitForFunction(
      "()=>document.querySelectorAll('table#main-table tbody tr').length===50",
      {"timeout":10000}
    ); await asyncio.sleep(0.3)

    # find col indices
    cols = await page.evaluate("""
      () => Array.from(
         document.querySelectorAll('table#main-table thead th')
      ).map(th=>th.textContent.trim())
    """)
    i_sym = cols.index("Symbol")
    i_vol = cols.index("Volume")
    i_cap = cols.index("Market Cap")
    i_pct = next(i for i,h in enumerate(cols) if "%" in h and "Change" in h)

    rows = await page.querySelectorAll("table#main-table tbody tr")
    data = []
    for r in rows:
        tds = await r.querySelectorAll("td")
        txt = [await page.evaluate("(el)=>el.textContent.trim()",td) for td in tds]
        def to_num(x):
            x = x.replace(",","").replace("M","e6").replace("K","e3")
            try: return float(x)
            except: return 0.0
        data.append({
            "Symbol":    txt[i_sym],
            "Volume":    to_num(txt[i_vol]),
            "Market Cap":to_num(txt[i_cap]),
            "PctChange": float(txt[i_pct].replace("%","") or 0.0)
        })
    await browser.close()
    return pd.DataFrame(data)

def scrape_top50():
    TODAY = datetime.now().strftime("%Y-%m-%d")
    df = asyncio.get_event_loop().run_until_complete(_scrape_top50())

    # fetch sector
    secs = []
    for s in df["Symbol"]:
        r = requests.get(f"https://stockanalysis.com/stocks/{s.lower()}/")
        soup = BeautifulSoup(r.text, "html.parser")
        span = soup.find("span",string=lambda t:t and t.strip()=="Sector")
        a    = span.find_next_sibling("a") if span else None
        secs.append(a.get_text(strip=True) if a else "")
        time.sleep(0.2)
    df["Sector"] = secs

    fn = os.path.join(DRIVE_PATH, f"top_{TODAY}_50.csv")
    df.to_csv(fn, index=False)
    print(f"☑️  Scraped → {fn}")

# ─── SELECTION + ORDER LOGIC ───────────────────────────────────────────────
def pick_and_buy():
    TODAY = datetime.now().strftime("%Y-%m-%d")
    fn = os.path.join(DRIVE_PATH, f"top_{TODAY}_50.csv")
    if not os.path.exists(fn):
        print("⚠️  no CSV yet, skipping buy")
        return

    df = pd.read_csv(fn)
    # normalize
    df["Pct_norm"] = (df["PctChange"] - df["PctChange"].min()) / (df["PctChange"].ptp()+1e-9)
    df["Cap_norm"] = (df["Market Cap"] - df["Market Cap"].min()) / (df["Market Cap"].ptp()+1e-9)
    df["Score"]    = PCT_WEIGHT*df["Pct_norm"] + (1-PCT_WEIGHT)*df["Cap_norm"]

    picks = df.nlargest(PICK_COUNT,"Score")["Symbol"].tolist()
    print("👉  BUY today:", picks)

    # fetch account buying power
    account = api.get_account()
    buying_power = float(account.cash) + float(account.buying_power)  # use cash+margin
    budget = buying_power * MAX_USAGE
    per_symbol = budget / PICK_COUNT

    for s in picks:
        # market order for $per_symbol notional
        try:
            api.submit_order(
                symbol=s,
                notional=round(per_symbol,2),
                side="buy",
                type="market",
                time_in_force="day"
            )
            print("  ✏️  Bought", s)
        except Exception as e:
            print("  ⚠️ buy failed", s, e)
        time.sleep(0.2)

def sell_all():
    positions = api.list_positions()
    print("👉  SELL all positions:", [p.symbol for p in positions])
    for p in positions:
        try:
            api.submit_order(
                symbol=p.symbol,
                qty=abs(int(float(p.qty))),
                side="sell",
                type="market",
                time_in_force="day"
            )
            print("  🗑️  Sold", p.symbol)
        except Exception as e:
            print("  ⚠️ sell failed", p.symbol, e)
        time.sleep(0.2)

# ─── SCHEDULE ─────────────────────────────────────────────────────────────
schedule.every().day.at(SCRAPE_TIME).do(scrape_top50)
schedule.every().day.at(BUY_TIME   ).do(pick_and_buy)
schedule.every().day.at(SELL_TIME  ).do(sell_all)

print("🕒  Scheduler started — Ctrl+C to stop")
while True:
    schedule.run_pending()
    time.sleep(10)


🕒  Scheduler started — Ctrl+C to stop


KeyboardInterrupt: 